In [7]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv(override=True)

True

In [2]:
def get_chatbot_response(client, model_name, messages, temperature=0):
    input_messages = []
    for message in messages:
        input_messages.append({"role": message["role"], "content": message["content"]})

    response = client.chat.completions.create(
        model=model_name,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000,
    )

    return response

In [9]:
client = OpenAI(
    # api_key=os.getenv("RUNPOD_TOKEN"),
    # base_url=os.getenv("RUNPOD_CHATBOT_URL"),
    api_key=os.environ.get("OPENAI_API_KEY")
)
# model_name = os.getenv("MODEL_NAME")
model_name = os.getenv("OPENAI_MODEL_NAME")

In [10]:
response = get_chatbot_response(client, model_name, [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
], temperature=0)

In [11]:
response.choices[0].message.content

'The capital of France is Paris.'

### Prompt Engineering

#### Structured output